In [246]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)

import sqlite3
import random
from faker import Faker
from flask import Flask, request, jsonify
from datetime import datetime

##### Creating database

In [248]:
conn = sqlite3.connect('student_performance.db', timeout=10)
cursor = conn.cursor()

##### Creating tables

In [250]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS students (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        age INTEGER
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS subjects (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        subject_name TEXT ,
        department TEXT
)
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS grades(
        id INTEGER PRIMARY KEY,
        student_id INTEGER,
        subject_id INTEGER,
        grade TEXT,
        date DATE,
        FOREIGN KEY (student_id) REFERENCES students(id),
        FOREIGN KEY (subject_id) REFERENCES subjects(id)
)
''')

conn.commit()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)

cursor.close()
conn.close()

Tables in the database: [('students',), ('subjects',), ('grades',)]


##### Generating tables data

In [252]:
faker = Faker()

students = []
students_no = 2000


for i in range(1, students_no + 1):
    name = faker.name()
    student = {
       # "id": i,
        "name" : name,
        "age": random.choice([None, random.randint(15, 25)]) if random.random() < 0.8 else random.randint(30, 80),
    }
    students.append(student)

In [253]:
subjects = []
students_no = 2000
subject_names = ["Mathematics", "Physics", "Chemistry", "Biology", "Geography", "Computer Science",
                 "History of Arts", "Philosophy", "Human of Arts", "Music", "Literature", "Creative Writing",
                "Accounting", "Economics", "Business Studies", "Finance", "Entrepreneurship", "International Business"]
departments = ["Science", "Arts", "Commerce"] 

for i, subject_name in enumerate(subject_names, start=1):
    subject = {
       # "id": i, 
        "subject_name": subject_name,
        "department": random.choice(departments),
    }
    subjects.append(subject)

In [254]:
grades = []
grades_list = ["A", "B", "C", "D", "F", None]

for i in range(1, students_no + 1):
    grade = {
        #"id": i,
        "student_id": random.randint(1, students_no + 1),
        "subject_id": random.randint(1, len(subjects)),
        "grade": random.choice(grades_list) if random.random() > 0.05 else f"Grade-{i}",  
        "date": None if random.random() > 0.95 else f"{random.randint(2010, 2025)}-{random.randint(1, 12):02}-{random.randint(1, 28):02}"
    }
    grades.append(grade)

In [255]:
students_df = pd.DataFrame(students)
subjects_df = pd.DataFrame(subjects)
grades_df = pd.DataFrame(grades)

*Replace NaN with None*

In [257]:
grades_df['grade'] = grades_df['grade'].where(grades_df['grade'].notna(), None)
students_df['age'] = students_df['age'].where(students_df['age'].notna(), None)

In [258]:
students_df.to_csv("students_dirty.csv", index=False)
subjects_df.to_csv("subjects_dirty.csv", index=False)
grades_df.to_csv("grades_dirty.csv", index=False)

In [259]:
students = pd.read_csv("students_dirty.csv")
subjects = pd.read_csv("subjects_dirty.csv")
grades = pd.read_csv("grades_dirty.csv")

##### Inserting the values into sqlite table

In [261]:
conn = sqlite3.connect('student_performance.db', timeout=10)
cursor = conn.cursor()

for _, row in students.iterrows():
    cursor.execute('''
        INSERT INTO students (name, age) 
        VALUES (?, ?)
    ''', (row['name'], row['age']))
conn.commit()

In [262]:
for _, row in subjects.iterrows():
    cursor.execute('''
        INSERT INTO subjects (subject_name, department)
        VALUES (?, ?)
    ''', (row['subject_name'], row['department']))
conn.commit()

In [263]:
for _, row in grades.iterrows():
    cursor.execute('''
        INSERT INTO grades (student_id, subject_id, grade, date)
        VALUES (?, ?, ?,?)
    ''', (row['student_id'], row['subject_id'], row['grade'], row['date']))
conn.commit()

In [264]:
cursor.close()
conn.close()

##### EDA

In [266]:
conn = sqlite3.connect('student_performance.db', timeout=10)
cursor = conn.cursor()

##### Average grade

In [268]:
cursor.execute('''
    SELECT 
        AVG(
            CASE grade
                WHEN 'A' THEN 4
                WHEN 'B' THEN 3
                WHEN 'C' THEN 2
                WHEN 'D' THEN 1
                WHEN 'F' THEN 0
                ELSE NULL
            END
        ) AS average_grade
        FROM grades;
''')

results1 = cursor.fetchall()

avg = ['average_grade']
df_avg = pd.DataFrame(results1, columns=avg)
df_avg

,average_grade
0,2.00998


##### Most and Least Popular Subjects

In [270]:
cursor.execute('''
    SELECT subject_name, COUNT(*) as enrollment_count
    FROM grades
    JOIN subjects ON subjects.id = grades.subject_id
    GROUP BY subject_name 
    ORDER BY enrollment_count DESC
    LIMIT 5;
''')

results2 = cursor.fetchall()

popular = ['subject_name', 'enrollment_count']
df_pop = pd.DataFrame(results2, columns=popular)
df_pop

,subject_name,enrollment_count
0,Chemistry,2095
1,History of Arts,1635
2,Economics,1493
3,International Business,1378
4,Music,718


##### Department-wise Student Distribution

In [272]:
cursor.execute('''
    SELECT 
    department, 
    COUNT(*) AS student_count
    FROM subjects
    JOIN grades ON grades.subject_id = subjects.id
    GROUP BY department
    ORDER BY student_count DESC;
''')

results3 = cursor.fetchall()

department = ['department', 'student_count']
df_dep = pd.DataFrame(results3, columns=department)
df_dep

,department,student_count
0,None,4190
1,Science,3396
2,Commerce,2997
3,Arts,1417


##### Top Scores in Each Subject

In [274]:
cursor.execute('''
    SELECT 
        subjects.subject_name, 
        COUNT(grades.grade) AS A_grades_count
    FROM grades
    JOIN subjects ON grades.subject_id = subjects.id
    WHERE grades.grade = 'A'
    GROUP BY subjects.subject_name
    ORDER BY subjects.subject_name
    LIMIT 8;
''')

results4 = cursor.fetchall()

scores = ['subject_name', 'A_grades_count']
df_scores = pd.DataFrame(results4, columns=scores)
df_scores

,subject_name,A_grades_count
0,Business Studies,116
1,Chemistry,341
2,Economics,252
3,Finance,80
4,History of Arts,300
5,Human of Arts,113
6,International Business,218
7,LiteratureCreative Writing,97


##### Grade Trends

In [276]:
cursor.execute('''
    SELECT STRFTIME('%Y', date) AS year, grade, COUNT(*) AS count
    FROM grades
    WHERE date IS NOT NULL
    GROUP BY year, grade
    ORDER BY year, grade;
''')

results5 = cursor.fetchall()

trends = ['year', 'grade', 'count']
df_trends = pd.DataFrame(results5, columns=trends)
df_trends

,year,grade,count
0,2010,None,141
1,2010,A,143
2,2010,B,155
3,2010,C,162
4,2010,D,133
...,...,...,...
863,2025,Grade-744,1
864,2025,Grade-780,1
865,2025,Grade-850,1
866,2025,Grade-890,1


##### Older Students’ Performance

In [282]:
cursor.execute('''
    SELECT 
    AVG(
        CASE grade
            WHEN 'A' THEN 4
            WHEN 'B' THEN 3
            WHEN 'C' THEN 2
            WHEN 'D' THEN 1
            WHEN 'F' THEN 0
            ELSE NULL
        END
    ) AS average_grade
    FROM grades
    JOIN students ON grades.student_id = students.id
    WHERE students.age > 30;
    ''')

results7 = cursor.fetchall()

above30 = ['average_grade']
df_above30 = pd.DataFrame(results7, columns=above30)
df_above30

,average_grade
0,2.040147
